In [1]:
import math
import random

import matplotlib.pyplot as plt

## Setup City

In [2]:
class City():
    
    def __init__(self, x, y):
        
        self.x = x
        self.y = y
        
    def get_distance(self, city):
        
        x_dist = abs(self.x - city.x)
        y_dist = abs(self.y - city.y)
        
        distance = np.sqrt((x_dist ** 2) + (y_dist ** 2))
        
    def __repr__(self):
        return '(' + str(self.x) + ',' + str(self.y) + ')'

## Define GA

In [3]:
class GeneticAlgorithm(object):
    """Generate GA for Traveling Salesman problem."""
    
    def __init__(self):
        
        self.name = 'GA for Traveling Salesman problem.'
        print('GA for Traveling Salesman problem successfully initialized.')
        
        self.distance = 0
        self.fitness_score = 0.0
    
    def generate_population(self, city_list, pop_size):
        
        population = []
        
        for i in range(0, pop_size):
            population.append(random.sample(city_list, len(city_list)))
            
        return population
    
    def get_route_distance(self, route):
        
        if self.distance == 0:
            
            path_distance = 0
            for i in range(0, len(route)):
                
                from_city = self.route[i]
                to_city = None
                
                if i+1 < len(self.route):
                    to_city = self.route[i + 1]
                else:
                    to_city = self.route[0]
                    
                path_distance += from_city.distance(to_city)
            self.distance = path_distance
            
        return self.distance
    
    def get_route_fitness(self, route):
        
        if self.fitness_score == 0:
            self.fitness_score = 1 / float(self.get_route_distance(route))
            
        return self.fitness_score
    
    def get_rank_routes(self, population):
        
        fitness_results = {}
        
        for i in range(0, len(population)):
            fitness_results[i] = self.get_route_fitness(population[i])
        return sorted(fitness_results.items(), key=operator.itemgetter(1), reverse=True)
    
    def crossover(self):
        pass
    
    def mutate(self):
        pass
    
    def create_next_generation(self):
        pass
    
    def reproduce(self):
        pass

---